In [2]:
import pandas as pd
import requests
import re

In [19]:
def crowl_content(id):
    url = 'https://www.dcard.tw/_api/posts/'+str(id)
    web = requests.get(url)
    jsn = web.json()
    return jsn['content']

In [102]:
url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100'

web = requests.get(url)

jsn = web.json()

content_list = []

for i in range(len(jsn)):
    ids = jsn[i]['id']
    content_list.append(crowl_content(ids))
    last = ids
    
for j in range(100):
    url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + str(last)
    web = requests.get(url)
    jsn = web.json()
    for i in range(len(jsn)):
        ids = jsn[i]['id']
        try:
            content_list.append(crowl_content(ids))
            last = ids
        except KeyError:
            last = ids

KeyboardInterrupt: 

In [103]:
len(content_list)

1635

In [65]:
import jieba
import pandas as pd
from collections import Counter
import math

In [66]:
jieba.load_userdict('testdic.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.159 seconds.
Prefix dict has been built succesfully.


In [88]:
def keyword(sentence):
#     cut the sentence with jieba
    article = sentence
    article_cut = jieba.lcut(article)
# only take words that length > 2
    filter_cut = []
    for w in article_cut:
        if len(w)>1:
            filter_cut.append(w)
# count 'tf' values
    c = Counter(filter_cut)
    total = len(set(filter_cut))
    df = pd.DataFrame(c.keys(),c.values())
    df['tf'] = df.index/total


# def log function
    def log(x):
        return math.log(x,10)

    total_content = len(content_list)
# count this word be metions in other articles times
    metion_list=[]
    for word in df[0]:
        metion = 0
        for a in content_list:
            if word in a:
                metion+=1
        if metion == 0:
            metion+=1
        metion_list.append(metion)
# count 'idf' values
    df['metion']=metion_list
    df['idf'] = df['metion'].apply(lambda x : log(total_content/x))
# count 'tfidf' values
    df['tfidf'] = df['tf']*df['idf']

    df = df.sort_values('tfidf',ascending=False)
    return df

In [100]:
kw = keyword('''考慮了好一陣子
終於去動了雙眼皮手術
我只諮詢過一家
我的訴求是希望有眼睛看起來有神自然就好
就算眼頭的地方被蒙古折遮住也沒關係
不一定要做到開放式的樣子

醫生很細心
會反覆模擬雙眼皮的形狀讓自己選擇
醫生的建議是小切口雙眼皮+取脂
切口的部分是有縫線的
所以之後還需要回診拆線

下面是不專業術後恢復照''')

In [101]:
kw

,0,tf,metion,idf,tfidf
3,雙眼皮,0.061224,6,2.662443,0.163007
2,切口,0.040816,1,3.440594,0.140432
2,醫生,0.040816,46,1.777836,0.072565
1,縫線,0.020408,1,3.440594,0.070216
1,去動,0.020408,1,3.440594,0.070216
1,拆線,0.020408,1,3.440594,0.070216
1,蒙古,0.020408,1,3.440594,0.070216
1,過一家,0.020408,1,3.440594,0.070216
1,取脂,0.020408,1,3.440594,0.070216
1,訴求,0.020408,2,3.139564,0.064073
